In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/home/lankenau/isotools/src')

In [3]:
import os
import logging
from isotools import Gene
from isotools import Transcriptome
from isotools import __version__ as isotools_version

# set up logging
logging.basicConfig(format="%(levelname)s:%(message)s", level=logging.INFO)
logger = logging.getLogger("isotools")
logger.info(f"This is isotools version {isotools_version}")

INFO:This is isotools version 0.3.5rc11


In [4]:

basepath = '/project/hfa_work/ENCODE/code/snakemake-pipeline/'
version = 2
tissue = 'brain'
path = os.path.join(basepath, f"results/isotools_v{version}/pkl/{tissue}.pkl")
isoseq = Transcriptome.load(path)

INFO:loading transcriptome from /project/hfa_work/ENCODE/code/snakemake-pipeline/results/isotools_v2/pkl/brain.pkl


In [5]:
classification = os.path.join(basepath, 'results/sqanti/isotools_v{version}/qc/{tissue}/brain_classification.txt')
isoseq.import_sqanti_classification(classification.format(version=version, tissue=tissue))

100%|██████████| 55811/55811 [00:19<00:00, 2924.26it/s]


In [6]:
isoseq.add_filter(tag='CAGE_SUPPORT', expression='sqanti_classification is not None and sqanti_classification["within_CAGE_peak"]', context='transcript', update=True)
isoseq.add_filter(tag='TSS_RATIO', expression='sqanti_classification is not None and sqanti_classification["ratio_TSS"] > 1.5', context='transcript', update=True)
isoseq.add_filter(tag='POLYA_SITE', expression='sqanti_classification is not None and sqanti_classification["within_polyA_site"]', context='transcript', update=True)
isoseq.add_filter(tag='POLYA_MOTIF', expression='sqanti_classification is not None and sqanti_classification["polyA_motif_found"]', context='transcript', update=True)

isoseq.add_filter(tag='SQANTI_SUPPORT', expression='CAGE_SUPPORT and TSS_RATIO and POLYA_SITE and POLYA_MOTIF', context='transcript', update=True)

In [7]:
next(isoseq.iter_transcripts(query='SQANTI_SUPPORT'))

(<isotools.gene.Gene object at 0x7f82835be870>,
 0,
 {'exons': [[np.int64(108876985), 108877228],
   [108885506, 108885578],
   [108896863, 108897085],
   [108897491, 108897627],
   [108897958, 108898101],
   [108898641, 108898765],
   [108898878, 108898994],
   [108901789, 108901945],
   [108903125, 108903234],
   [108904124, 108904254],
   [108914337, 108914408],
   [108918612, 108918789],
   [108922416, 108922576],
   [108923999, 108924214],
   [108929700, np.int64(108930395)]],
  'strand': '+',
  'coverage': {'ENCSR700EBI': 10, 'ENCSR425HFS': 1, 'ENCSR463IDK': 10},
  'TSS': {'ENCSR700EBI': {108876981: 1,
    108876984: 1,
    108876985: 2,
    108876986: 1,
    108876988: 2,
    108876990: 1,
    108877106: 1,
    108877146: 1},
   'ENCSR425HFS': {108876984: 1},
   'ENCSR463IDK': {108876984: 5,
    108876985: 1,
    108876987: 2,
    108877098: 1,
    108877168: 1}},
  'PAS': {'ENCSR700EBI': {108930396: 7,
    108931627: 1,
    108930404: 1,
    108930391: 1},
   'ENCSR425HFS': {10

In [8]:
count = 0
for transcript in isoseq.iter_transcripts(query='SQANTI_SUPPORT'):
    count += 1
count

11661

In [10]:
coordination_results = isoseq.coordination_test(
    event_type=["TSS", "PAS"],
    min_total=50,
    min_alt_fraction=.05,
    transcript_filter="SQANTI_SUPPORT",
)
coordination_results

,gene_id,gene_name,strand,eventA_type,eventB_type,eventA_start,eventA_end,eventB_start,eventB_end,pvalue,...,dcPSI_AB,dcPSI_BA,priA_priB,priA_altB,altA_priB,altA_altB,priA_priB_transcript_ids,priA_altB_transcript_ids,altA_priB_transcript_ids,altA_altB_transcript_ids
0,ENSG00000142875.21,PRKACB,+,TSS,PAS,84078084,84078371,84204481,84205294,8.387972e-15,...,0.458757,0.491525,29,2,0,28,"[4, 3, 2]",[0],[],[1]
1,ENSG00000142875.21,PRKACB,+,TSS,PAS,84078084,84078371,84235179,84238491,8.387972e-15,...,-0.458757,-0.443958,0,28,29,2,[],[1],"[4, 3, 2]",[0]
2,ENSG00000142875.21,PRKACB,+,TSS,PAS,84164341,84164422,84204481,84205294,5.356978e-02,...,-0.474576,-0.084746,26,5,28,0,"[4, 0]","[3, 2]",[1],[]
3,ENSG00000142875.21,PRKACB,+,TSS,PAS,84164341,84164422,84235179,84238491,5.356978e-02,...,0.474576,0.076545,28,0,26,5,[1],[],"[4, 0]","[3, 2]"
4,ENSG00000142875.21,PRKACB,+,TSS,PAS,84164694,84165013,84204481,84205294,1.225092e-10,...,-0.474576,-0.406780,7,24,28,0,"[3, 0, 2]",[4],[1],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,ENSG00000172465.14,TCEAL1,+,TSS,PAS,103629476,103629573,103629884,103630946,3.526644e-06,...,0.607843,0.334314,31,0,9,11,[1],[],"[2, 0, 4]",[3]
250,ENSG00000133142.19,TCEAL4,+,TSS,PAS,103585517,103585624,103586648,103587725,1.274413e-63,...,-0.107557,-0.497652,8,570,84,50,[6],[1],"[2, 5]","[3, 0, 4]"
251,ENSG00000133142.19,TCEAL4,+,TSS,PAS,103585517,103585624,103586653,103587725,1.274413e-63,...,0.107557,0.115373,84,50,8,570,"[2, 5]","[3, 0, 4]",[6],[1]
252,ENSG00000133142.19,TCEAL4,+,TSS,PAS,103585517,103585683,103586648,103587725,1.742996e-59,...,0.720889,0.473419,570,8,54,80,[1],[6],"[3, 5, 0, 4]",[2]
